In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
import pandas as pd

import preprocessor as pr
from tensorflow.contrib import rnn

from konlpy.tag import Twitter

In [2]:
# TODO 6.1 : TODO 1 ~ 5 까지 데이터 Load (using by Pickle)
vocab_dict = pr.load_pickle('./data/question_dict.pickle')
operator_dict = pr.load_pickle('./data/operator_dict.pickle')

In [3]:
input_dim  = x_num_classes  = n_words = len(vocab_dict)               # 모두 2048 
output_dim = y_num_classes = hidden_size = len(operator_dict)         # 모두   99
# lstm_size: Number of units in the hidden layers in the LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512. etc
lstm_size = hidden_dim = 512
# lstm_layers: Number of LSTM layers in the network. I'd start with 1, then add more if I'm underfitting.
lstm_layers = 1
# batch_size: The number of reviews to feed the network in one training pass. Typically this should be set as high as you can go without running out of memory

epochs        = 10  # 1 epoch = one forward pass and one backward pass of all the  training examples.
learning_rate = 0.02
batch_size    = 100  # batch size = the number of training examples in one forward/backward pass.

In [4]:
dataX = []
dataY = []
twitter = Twitter()

In [5]:
df_sop = pd.read_csv('./data/question_operator_set_1500.csv')

for idx, value in enumerate(df_sop['question']):
    dataX.append([vocab_dict[question] for question in twitter.morphs(value)]) 
    
for idx, value in enumerate(df_sop['operator']):
    dataY.append(operator_dict[value])

In [6]:
print("shape of dataX : {}".format(np.array(dataX).shape)) # (1500, ) : 가변길이이기 때문에 정확하게 표현되지 않음
print("shape of dataY : {}".format(np.array(dataY).shape)) # (1500, )

shape of dataX : (1500,)
shape of dataY : (1500,)


In [7]:
sequence_length_list = [len(val) for idx, val in enumerate(dataX)]
sequence_length = max_len = max(sequence_length_list)

In [8]:
# left Zero-padding, 단 Zero-Padding시에도 dynamic_rnn 사용함.
# Many to One 모델 적용으로 left Zero-padding
for idx, val in enumerate(dataX):
    if len(dataX[idx]) < max_len:
        for ind in range(0, max_len - len(dataX[idx])):
            # dataX[idx].append(0)  right zero-padding
            dataX[idx].insert(0, 0) # left zero-padding

In [9]:
print("shape of dataX : {}".format(np.array(dataX).shape)) # (1500, 50) : left Zero-padding으로 (1500, 50)
print("shape of dataY : {}".format(np.array(dataY).shape)) # (1500, )

shape of dataX : (1500, 50)
shape of dataY : (1500,)


In [10]:
split_train_frac = 0.8

split_index = int(split_train_frac * len(dataX))

train_x, val_x = np.array(dataX[:split_index]), np.array(dataX[split_index:]) 
train_y, val_y = np.array(dataY[:split_index]), np.array(dataY[split_index:])

split_test_frac = 0.5
split_index = int(split_test_frac * len(val_x))

val_x, test_x = np.array(val_x[:split_index]), np.array(val_x[split_index:])
val_y, test_y = np.array(val_y[:split_index]), np.array(val_y[split_index:])

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(1200, 50) 
Validation set: 	(150, 50) 
Test set: 		(150, 50)
label set: 		(1200,) 
Validation label set: 	(150,) 
Test label set: 	(150,)


In [11]:
# batch-size = 1을 위한 훈련 데이터 변경 (iterations : 10)
#train_x, train_y = train_x[: 10], train_y[: 10]

In [12]:
# 동일 데이터를 학습시켜보기
'''
tmp_x, tmp_y = [], []
for idx in range(10):
    tmp_x, tmp_y = np.append(tmp_x, train_x), np.append(tmp_y, train_y)
train_x, train_y = np.reshape(tmp_x, [-1, 50]), np.reshape(tmp_y, [-1])
'''

'\ntmp_x, tmp_y = [], []\nfor idx in range(10):\n    tmp_x, tmp_y = np.append(tmp_x, train_x), np.append(tmp_y, train_y)\ntrain_x, train_y = np.reshape(tmp_x, [-1, 50]), np.reshape(tmp_y, [-1])\n'

In [13]:
# 동일 데이터를 통해 validation
val_x = train_x[1050 :, :]
val_y = train_y[1050 :]

In [14]:
val_y.shape

(150,)

In [15]:
# output 정규화 >> tf.conrib.layer.bat_norm 사용 방법 확인해볼 것
# train_y = train_y / max(train_y) - min(train_y)
# val_y = val_y / max(val_y) - min(val_y)
# test_y = test_y / max(test_y) - min(test_y)

In [16]:
# Create the graph object
tf.reset_default_graph()

# Logging 설정 - 
tf.logging.set_verbosity(tf.logging.INFO)

with tf.name_scope('Input'):
    X = tf.placeholder(tf.int32, [None, None], name="X")
    Y = tf.placeholder(tf.int32, [None], name="Y")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")

In [17]:
print(X)
print(Y)

Tensor("Input/X:0", shape=(?, ?), dtype=int32)
Tensor("Input/Y:0", shape=(?,), dtype=int32)


In [18]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with tf.name_scope("Embeddings"):
    embedding_x = tf.Variable(tf.random_uniform((x_num_classes, embed_size), -1, 1))
    embed_x = tf.nn.embedding_lookup(embedding_x, X)

#X_one_hot = tf.one_hot(X, x_num_classes)
Y_one_hot = tf.one_hot(Y, y_num_classes, 1, 0)
#Y_one_hot_rs = tf.reshape(Y_one_hot, [-1, 99])

In [19]:
print("embedding_x : " + str(embedding_x)) 
print("embed_x : " + str(embed_x)) # (?, ?, 300)
print("Y_one_hot : " + str(Y_one_hot)) # (?, 99)

embedding_x : Tensor("Embeddings/Variable/read:0", shape=(2047, 300), dtype=float32)
embed_x : Tensor("Embeddings/embedding_lookup:0", shape=(?, ?, 300), dtype=float32)
Y_one_hot : Tensor("one_hot:0", shape=(?, 99), dtype=int32)


In [20]:
def lstm_cell():
    # Your basic LSTM cell
    #lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    # Add dropout to the cell
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

In [21]:
with tf.name_scope("RNN_layers"):
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [22]:
print(initial_state) # shape=(100, 256)

(LSTMStateTuple(c=<tf.Tensor 'RNN_layers/zeros:0' shape=(100, 512) dtype=float32>, h=<tf.Tensor 'RNN_layers/zeros_1:0' shape=(100, 512) dtype=float32>),)


In [23]:
with tf.name_scope("RNN_forward"):
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, embed_x, initial_state=initial_state)
#cell = rnn.BasicLSTMCell(hidden_dim, state_is_tuple = True)
#initial_state = cell.zero_state(batch_size, tf.float32)
#outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, sequence_length=sequence_length_list, dtype=tf.float32)
#outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)
#rnn_outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

In [24]:
final_state, Y_one_hot, rnn_outputs  # shape=(1, 512) shape=(1, 512) shape=(1, ?, 512)

((LSTMStateTuple(c=<tf.Tensor 'RNN_forward/rnn/while/Exit_2:0' shape=(100, 512) dtype=float32>, h=<tf.Tensor 'RNN_forward/rnn/while/Exit_3:0' shape=(100, 512) dtype=float32>),),
 <tf.Tensor 'one_hot:0' shape=(?, 99) dtype=int32>,
 <tf.Tensor 'RNN_forward/rnn/transpose:0' shape=(100, ?, 512) dtype=float32>)

In [25]:
#funciton to transform gradients
def transform_grad(grad, decay=1.0):
    #return decayed gradient
    return decay*grad

In [26]:
with tf.name_scope('predictions'):
    fc_output = tf.contrib.layers.fully_connected(rnn_outputs[:, -1], y_num_classes, activation_fn=None)
    tf.summary.histogram('logits', fc_output)
    
with tf.name_scope('cost'):
    #cost = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=Y_one_hot)
    cost = tf.nn.softmax_cross_entropy_with_logits(logits=fc_output, labels=tf.to_float(Y_one_hot))
    cost = tf.reduce_sum(cost)
    tf.summary.scalar('cost', cost)
    
# Defintion of cross entropy loss function
# D(y, y^hat) = - Sum(y element-wise multiplcation log (y^hat))
    
    calculated_loss = -tf.reduce_sum(tf.to_float(Y_one_hot) * tf.log(tf.nn.softmax(fc_output)))

with tf.name_scope('train'):
    
    #opt = tf.train.GradientDescentOptimizer(learning_rate)
    
    # gradient variable list = [ (gradient,variable) ]
    #gv = opt.compute_gradients(cost)
    
    # transformed gradient variable list = [ (T(gradient),variable) ]
    #decay = 0.9 # decay the gradient for the sake of the example
    #tgv = [ (g, v) for (g,v) in gv] #list [(grad,var)]
    #tgv = [ (T(g,decay=decay), v) for (g,v) in gv] #list [(grad,var)]
    
    # apply transformed gradients (this case no transform)
    #apply_transform_op = opt.apply_gradients(tgv)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

merged = tf.summary.merge_all()

In [27]:
# 
#opt = tf.train.GradientDescentOptimizer(learning_rate)

# gradient variable list = [ (gradient,variable) ]
#gv = opt.compute_gradients(y,[x])

# transformed gradient variable list = [ (T(gradient),variable) ]
#decay = 0.9 # decay the gradient for the sake of the example
#tgv = [ (T(g,decay=decay), v) for (g,v) in gv] #list [(grad,var)]

# apply transformed gradients (this case no transform)
#apply_transform_op = opt.apply_gradients(tgv)

In [28]:
with tf.name_scope('validation'):
    correct = tf.equal(tf.argmax(fc_output,1), tf.argmax(Y_one_hot,1))
    accuracy = tf.reduce_mean(tf.cast(correct,'float'))

In [29]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [30]:
print("==== Feed Data의 차원 ====")
print("dataX : " + str(np.array(dataX).shape))
print("dataY : " + str(np.array(dataY).shape))

print("\n==== 입력 - 출력 텐서의 차원 변화 ====")
print("placeholder X : " + str(X.get_shape()))
print("placeholder Y : " + str(Y.get_shape()))
print("embedding_x : " + str(embedding_x))
print("embed_x : " + str(embed_x))
print("rnn_outputs : " + str(rnn_outputs.get_shape()))
# print("predictions : " + str(predictions.get_shape()))

==== Feed Data의 차원 ====
dataX : (1500, 50)
dataY : (1500,)

==== 입력 - 출력 텐서의 차원 변화 ====
placeholder X : (?, ?)
placeholder Y : (?,)
embedding_x : Tensor("Embeddings/Variable/read:0", shape=(2047, 300), dtype=float32)
embed_x : Tensor("Embeddings/embedding_lookup:0", shape=(?, ?, 300), dtype=float32)
rnn_outputs : (100, ?, 512)


In [31]:
# with graph.as_default():
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/tb/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/tb/test', sess.graph)
    
    statelist = []
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for idx, (x, y) in enumerate(get_batches(train_x, train_y, batch_size)):
            feed = {X: x, Y: y, keep_prob: 0.8, initial_state: state}
            #embed_xx, yone, rnn_out, logit, loss, _ = sess.run([embed_x, Y_one_hot, rnn_outputs, logits, cost, optimizer], feed_dict=feed)
            summary, losses, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)
            #summary, losses, state, _, grad = sess.run([merged, cost, final_state, apply_transform_op, gv], feed_dict=feed)
            
            yone, logit, c_loss = sess.run([Y_one_hot, fc_output, calculated_loss], feed_dict=feed)
            '''
            if ii == 1:
                print("shape of x : {}".format(x.shape))
                print("shape of y : {}".format(y.shape))
                print("shape of embed_xx : {}".format(embed_xx .shape))
                print("shape of yone : {}".format(yone.shape))
                print("shape of rnn_outputs : {}".format(rnn_out.shape))
                print("shape of rnn_outputs_last_step : {}".format(rnn_out[:, -1].shape))
                #print("shape of states : {}".format(np.array(final).shape))
            '''
            # 
            statelist.append(state)
            
            train_writer.add_summary(summary, iteration)

            #if iteration%1==0:
            print("Epoch: {}/{}".format(e, epochs), "Iteration: {}".format(iteration), "Train loss: {:.3f}".format(losses))
            #print("dimension of grad: {}".format(np.array(grad).shape))
            #print("sum of grad: {}".format(sum(np.array(grad)[idx])))
            #print("sum of calculated loss : {}".format(c_loss))
            
            if iteration%5==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {X: x, Y: y, keep_prob: 1, initial_state: val_state}
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                    
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
                
            iteration +=1
            test_writer.add_summary(summary, iteration)
            saver.save(sess, "checkpoints/sentiment_manish.ckpt")
    saver.save(sess, "checkpoints/sentiment_manish.ckpt")                

Epoch: 0/10 Iteration: 1 Train loss: 459.777
Epoch: 0/10 Iteration: 2 Train loss: 438.459
Epoch: 0/10 Iteration: 3 Train loss: 436.621
Epoch: 0/10 Iteration: 4 Train loss: 439.046
Epoch: 0/10 Iteration: 5 Train loss: 441.913
Val acc: 0.060
Epoch: 0/10 Iteration: 6 Train loss: 442.190
Epoch: 0/10 Iteration: 7 Train loss: 432.781
Epoch: 0/10 Iteration: 8 Train loss: 441.055
Epoch: 0/10 Iteration: 9 Train loss: 435.522
Epoch: 0/10 Iteration: 10 Train loss: 438.550
Val acc: 0.070
Epoch: 0/10 Iteration: 11 Train loss: 431.004
Epoch: 0/10 Iteration: 12 Train loss: 434.464
Epoch: 1/10 Iteration: 13 Train loss: 425.869
Epoch: 1/10 Iteration: 14 Train loss: 423.040
Epoch: 1/10 Iteration: 15 Train loss: 423.504
Val acc: 0.080
Epoch: 1/10 Iteration: 16 Train loss: 431.143
Epoch: 1/10 Iteration: 17 Train loss: 430.595
Epoch: 1/10 Iteration: 18 Train loss: 433.237
Epoch: 1/10 Iteration: 19 Train loss: 432.088
Epoch: 1/10 Iteration: 20 Train loss: 437.334
Val acc: 0.070
Epoch: 1/10 Iteration: 21 Tra

KeyboardInterrupt: 